In [1]:
import matplotlib
import matplotlib.path as mplPath
import numpy as np
import matplotlib.pyplot as plt
import gmplot
import itertools
import requests
import copy
import ast
import re
import json
import csv
from IPython.core.display import display
from IPython.lib.display import IFrame
import os
%matplotlib inline
#%matplotlib auto

## Start here

In [2]:
class BasicFn(object):
    # эти функции нужно будет закинуь в класс
    def readUrl(self, url):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'\
                + ' AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
        }
        page = requests.get(url, headers=headers)
        return page.content
    def uniqueArr(self, arr):
        seen = set()
        seen_add = seen.add
        return [x for x in arr if not (x in seen or seen_add(x))]
    

In [8]:
class DataParser(BasicFn):
    def __init__(self, fileName):
        super(DataParser, self).__init__()
        self.postalsInfo = self.getPostalZones(fileName)
    # parsing our data points
    def getPostalZones(self, fileName):
        postalZones = []
        with open(fileName, 'rb') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
            for row in spamreader:
                if(spamreader.line_num == 1):
                    continue
                coords = row[0].replace('POLYGON ', '').replace('(', '').replace(")", '').split(',')
                x, y = [], []
                for el in coords:
                    el = el.split(' ')
                    x.append(float(el[-1]))
                    y.append(float(el[0]))
                point = {
                    'postal': row[-2],
                    'dest_lat': x,
                    'dest_lon': y
                }
                postalZones.append(point)
        return postalZones
    # parsing isoscope
    def parseIsoScope(self, lat, lon, sec=9000):
        urlIso = """
                https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json?app_id=bVQHRXUn6uNHP3B24bdt
                &app_code=mwkokcyyoCIfExsmQq0qIg
                &departure=2016-12-26T00:00:00
                &mode=fastest;car;traffic:enabled
                &rangetype=time
                &start=geo!""" + str(lat) +','+ str(lon) +"""
                &range=""" + str(sec) + """
                &singlecomponent=false
                &resolution=10
                &quality=3
                &requestId=2a770ce3106933a0760d2b24afccd098
                &jsonCallback=callback_2a770ce3106933a0760d2b24afccd098&callback=__jp240"""
        urlIso = urlIso.replace('\n', '').replace('\t', '').replace(' ', '')
        with open('cachedData.csv', 'ab+') as f:
            reader = csv.reader(f, delimiter=',', quotechar='"')
            for row in reader:
                if float(row[0])==lat and float(row[1])==lon:
                    return ast.literal_eval(row[2]), ast.literal_eval(row[3])
            print 'Requesting...
            print 'x:', float(row[0]), 'y:', float(row[1])
            writer = csv.writer(f)
            getReq = self.readUrl(urlIso)
            page = json.loads(re.sub(r'^[^{]*', '', getReq)[:-3])
            coord = page['response']['isoline'][0]['component'][0]['shape']
            x, y = [], []
            for i in coord:
                d = i.split(',')
                x.append(float(d[0]))
                y.append(float(d[1]))
            writer.writerow([lat, lon, x, y])
            return x, y
        
    # getting office locations from the file
    def office_locations(self, fileName):
        result = {}
        with open(fileName, 'rb') as csvfile:
            office_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
            for row in office_reader:
                if(office_reader.line_num == 1):
                    continue
                name = row[-3]
                if not name in result:
                    result[name] = {
                        'name': name,
                        'centerX': float(row[-2]),
                        'centerY': float(row[-1]),
                        'postal_collision': [row[-5]]
                    }
                elif not row[-5] in result[name]['postal_collision']:
                    result[name]['postal_collision'].append(row[-5])
        return [result[x] for x in result]

SyntaxError: EOL while scanning string literal (<ipython-input-8-58b8ddd7ff7c>, line 47)

In [9]:
class OfficeAreas(DataParser):
    def __init__(self, postalsFile):
        super(OfficeAreas, self).__init__(postalsFile)

    def convertOfficeFile(self, officeFile):
        offices = self.office_locations(officeFile)
        for i, office in enumerate(offices):
            x, y = self.parseIsoScope(office['centerX'], office['centerY'])
            offices[i]['dest_lat'] = x
            offices[i]['dest_lon'] = y
        return offices
    def draw(self, offices, postal_areas, **kwargs):
        fileName = kwargs.get('fileName',"map.html")
        show_used = kwargs.get('show_used', False)
                
        usedPostals = list(itertools.chain.from_iterable([x['postal_collision'] for x in offices]))
        usedPostals = self.uniqueArr(usedPostals)
        
        # saving only needed postal_areas
        if not show_used:            
            elements = copy.deepcopy(postal_areas)
            postal_areas = []
            for el in elements:
                if el['postal'] in usedPostals:
                    postal_areas.append(el)

        # getting "focusing point" - where map will focus and initiating map view
        x = y = None
        for area in postal_areas:
            avgX = np.average(area['dest_lat'])
            avgY = np.average(area['dest_lon'])
            if x is None:
                x, y = avgX, avgY
            else:
                x, y = np.average([x, avgX]), np.average([y, avgY])
                
        zoomLevel = kwargs.get('zoomLevel', 12)
        gmap = gmplot.GoogleMapPlotter(x, y, zoomLevel)
        
        unUsedColor = 'cornflowerblue'
        usedColor = 'forestgreen'
        officeColors = 'red'
        
        # drawing postal code zones
        markers = []
        for area in postal_areas:
            # plot || polygon
            if (area['postal'] in usedPostals) or show_used == True:
                gmap.plot(area['dest_lat'], area['dest_lon'],\
                              usedColor if area['postal'] in usedPostals else unUsedColor, edge_width=3)
                if 'centerX' in area:
                    markers.append({'centerX': area['centerX'],
                                    'centerY': area['centerY'],
                                    'title': area['postal'], 
                                    'office': False})
                else:
                    markers.append({'centerX': np.average(area['dest_lat']),
                                    'centerY': np.average(area['dest_lon']),
                                    'title': area['postal'], 
                                    'office': False})
        
        # in case we draw "taken zone", currently we don't
        for office in offices:
            # plot || polygon
            if 'dest_lat' in office:
                gmap.polygon(office['dest_lat'], office['dest_lon'], officeColors, edge_width=3)
            markers.append({'centerX': np.average(office['centerX']),
                            'centerY': np.average(office['centerY']),
                            'title': office['name'],
                            'office': True})            
            
        gmap.draw(fileName)
        # проблема в правах доступа и маркер не подтягивает(он находится в папке с библиотекой, а Chrome не дает к ней доступ)
        data = []
        with open(fileName, 'r') as f:
            #https://developers.google.com/maps/documentation/javascript/infowindows
            jsCode = """
            addMarkers(map);
            }            
            var markersList = """ + json.dumps(markers) + """;
            // https://developers.google.com/maps/documentation/javascript/reference#MarkerLabel
            function newMarker(title, x, y, office=false){
                var image = 'http://i.imgur.com/yFFV96y.jpg';
                return new google.maps.Marker({
                        label: {text: title, fontWeight: '900'},
                        icon: office ? '' : image,
                        position: new google.maps.LatLng(x, y)
                    });
            }
            function addMarkers(map){
                // showing postal codes that are clusterable
                var markers = markersList.filter((v)=>!v.office).map((v)=>newMarker(v.title, v.centerX, v.centerY));
                var markerCluster = new MarkerClusterer(map, markers,
                    {imagePath: 'https://developers.google.com/maps/documentation/javascript/examples/markerclusterer/m', maxZoom: 10}
                );
                // showing office locations(unclusterable)
                markersList.filter((v)=>v.office).forEach((v)=>newMarker(v.title, v.centerX, v.centerY, true).setMap(map));
            }
            </script>
            <!--как-то неверно его загружать в самом конце, но пока что работает(теоретически могут быть траблы, если не успеет подгрузиться)-->
            <script src="https://developers.google.com/maps/documentation/javascript/examples/markerclusterer/markerclusterer.js"></script>
            """
            data = f.read().replace('}\n</script>', jsCode)
            
        os.remove(fileName)
        with open(fileName, 'a') as f:
            f.write(data)
        display(IFrame(fileName, height = 500, width = "90%"))

In [10]:
all_postal_areas = 'geocsv_rigth.csv'
office_data = '2.csv'
done = OfficeAreas(all_postal_areas)
# throw additional function - in case we need multiple calculations for diff offices, so that we don't recreate class
offices = done.convertOfficeFile(office_data)
# 'geocsv_rigth.csv
done.draw(offices, done.postalsInfo, show_used=False, fileName='mymap.html')